In [18]:
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from getpass import getpass
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, PromptTemplate
from langchain import hub
from langchain_openai import ChatOpenAI
from graphdatascience import GraphDataScience

/Users/nathansmith/anaconda3/envs/chatbot/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
from langchain_google_vertexai import VertexAIEmbeddings
from google.auth.credentials import Credentials

In [3]:
openai_api_key = getpass()

 ········


In [21]:
neo4j_password = getpass()

 ········


In [28]:
gcloud_token = getpass()

 ········


In [22]:
neo4j_uri = "neo4j+s://bc9f751a.databases.neo4j.io"
neo4j_user = "neo4j"
gds = GraphDataScience(neo4j_uri, auth=(neo4j_user, neo4j_password))

In [7]:
!export PATH="/Users/nathansmith/google-cloud-sdk/bin:/Users/nathansmith/google-cloud-sdk/bin:/Users/nathansmith/anaconda3/envs/chatbot/bin:/Users/nathansmith/anaconda3/condabin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin:/Library/TeX/texbin"; gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=9zYfYLCyviejwZXSImJJIMgUUhkEEt&access_type=offline&code_challenge=J1-GJ1BHaGErnA0mEc2XWKy72HCtDr1Am2eZQp6If1Q&code_challenge_method=S256


You are now logged in as [nathan.smith@neo4j.com].
Your current project is [neo4j-ps-202001].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [40]:
!cd /Users/nathansmith/google-cloud-sdk/bin

In [8]:
!echo $PATH

/Users/nathansmith/anaconda3/bin:/Users/nathansmith/anaconda3/condabin:/usr/bin:/bin:/usr/sbin:/sbin


In [41]:
!gcloud auth login

zsh:1: command not found: gcloud


In [42]:
!pwd

/Users/nathansmith/Documents/blogs/RAG/Agent Neo


In [ ]:
summary_query = """
MATCH (node)<-[:IN_GROUP]-()<-[:HAS_TOPIC]-(d)
WITH d, max(score) AS score 
RETURN d.overview AS text, score, d{.title, .tagline} AS metadata
"""

summary_vectorstore = Neo4jVector.from_existing_index(
    VertexAIEmbeddings(),
    index_name="theme_group_summary_vectors",
    retrieval_query=summary_query,
    url=neo4j_uri,
    username=neo4j_user,
    password=neo4j_password
)

In [32]:
def query_topic_groups(query):
    result = gds.run_cypher(
    """ 
    WITH genai.vector.encode($query, "VertexAI", { token: $token, projectId: $project }) AS queryVector
    CALL db.index.vector.queryNodes("topic_group_summary_embeddings", 5, queryVector) YIELD node, score
    MATCH (node)<-[:IN_GROUP]-()<-[:HAS_TOPIC]-(d)
    WITH d, max(score) AS score 
    RETURN d.text AS text, score, d{.url} AS metadata""",
        {"query":query, "token": gcloud_token, "project": "neo4j-ps-202001"})
    return result

In [51]:
topic_results = query_topic_groups("How does the Cypher MERGE statement work?")

In [52]:
print("|".join(topic_results['text'].to_list()))

Transaction A Transaction B The following error message is thrown: The Cypher clause MERGE takes locks out of order to ensure the uniqueness of the data, and this may prevent Neo4j’s internal sorting operations from ordering transactions in a way that avoids deadlocks. When possible, you are, therefore, encouraged to use the Cypher clause CREATE instead, which does not take locks out of order. Deadlock handling in code|apoc.merge.node Procedure apoc.merge.node(labels [String], identProps Map<String, Any>, props Map<String, Any>, onMatchProps Map<String, Any>) - merges the given node(s) with the given dynamic labels. Signature Input parameters labels LIST? OF STRING? null identProps MAP? null props MAP? {} onMatchProps MAP? {} Output parameters node NODE? Usage Examples This procedure provides a more flexible way of merging nodes than Cypher’s MERGE clause.|15 20000 export-cypher-format-create.cypher The creation of all graph entities uses the Cypher CREATE clause. If those entities may

The MERGE clause in Cypher either matches existing node patterns in the graph and binds them, or if not present, creates new data and binds that. It acts as a combination of MATCH and CREATE, allowing specific actions depending on whether the specified data was matched or created. It guarantees the existence of the MERGE pattern but not uniqueness. To ensure uniqueness of nodes with certain properties, a property uniqueness constraint should be used. If there are multiple matches, all of them will be passed on to later stages of the query. It is recommended to use bound nodes in the MERGE pattern, especially when relationship uniqueness constraints exist.

In [34]:
def query_documents(query):
    result = gds.run_cypher(
    """ 
    WITH genai.vector.encode($query, "VertexAI", { token: $token, projectId: $project }) AS queryVector
    CALL db.index.vector.queryNodes("document-embeddings", 5, queryVector) YIELD node, score
    RETURN node.text AS text, score, node{.url} AS metadata""",
        {"query":query, "token": gcloud_token, "project": "neo4j-ps-202001"})
    return result

In [44]:
doc_results = query_documents("How does the Cypher MERGE statement work?")

In [50]:
print("|".join(doc_results['text'].to_list()))

MERGE Introduction The MERGE clause either matches existing node patterns in the graph and binds them or, if not present, creates new data and binds that. In this way, it acts as a combination of MATCH and CREATE that allows for specific actions depending on whether the specified data was matched or created.|When using MERGE on full patterns, the behavior is that either the whole pattern matches, or the whole pattern is created. MERGE will not partially use existing patterns. If partial matches are needed, this can be accomplished by splitting a pattern into multiple MERGE clauses.|Under concurrent updates, MERGE only guarantees the existence of the MERGE pattern, but not uniqueness. To guarantee uniqueness of nodes with certain properties, a property uniqueness constraint should be used. See Using property uniqueness constraints with MERGE. Similar to MATCH, MERGE can match multiple occurrences of a pattern. If there are multiple matches, they will all be passed on to later stages of 

The MERGE clause in Cypher either matches existing node patterns in the graph and binds them or, if not present, creates new data and binds that. It acts as a combination of MATCH and CREATE, allowing for specific actions depending on whether the specified data was matched or created. When using MERGE on full patterns, the behavior is that either the whole pattern matches, or the whole pattern is created. MERGE will not partially use existing patterns. If there are multiple matches, they will all be passed on to later stages of the query. Under concurrent updates, MERGE only guarantees the existence of the MERGE pattern, but not uniqueness. To guarantee uniqueness of nodes with certain properties, a property uniqueness constraint should be used.

In [38]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a Neo4j expert providing information about Neo4j software.
                  Be as helpful as possible and return as much information as possible.
                  Do not answer any questions that do not relate to Neo4j.
                  Answer any questions based solely on the context below:
                  <context>
                  {context}
                  </context>"""),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("user", "{input}"),
])

In [18]:
movie_info_template= "overview: {page_content}, title: {title}, tagline: {tagline}"
document_prompt = PromptTemplate(input_variables=["page_content", "url"],
                                 template=movie_info_template
                                 )


In [44]:
summary_retriever = summary_vectorstore.as_retriever()
movie_retriever = movie_vectorstore.as_retriever()

retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

combine_docs_chain = create_stuff_documents_chain(
    ChatOpenAI(temperature=0, openai_api_key=openai_api_key), prompt, document_prompt=document_prompt

)
summary_chain = create_retrieval_chain(summary_retriever, combine_docs_chain)
movie_chain = create_retrieval_chain(movie_retriever, combine_docs_chain)